# 🚀 LangGraph QuickStart 튜토리얼

## 📚 개요

이 튜토리얼은 LangGraph의 핵심 개념과 기능을 단계별로 학습하는 종합 가이드입니다. 기본 챗봇 구축부터 시작하여 도구 통합, 메모리 관리, Human-in-the-Loop, 상태 커스터마이징, 타임 트래블까지 LangGraph의 강력한 기능들을 체계적으로 익힐 수 있습니다.

## 🎯 학습 목표

1. **기본 챗봇 구축**: StateGraph를 사용한 기본적인 대화형 AI 구현
2. **도구 통합**: 외부 API와 도구를 활용한 챗봇 기능 확장
3. **메모리 관리**: 체크포인팅을 통한 대화 상태 유지
4. **Human-in-the-Loop**: 인간 개입이 필요한 워크플로우 구현
5. **상태 커스터마이징**: 복잡한 비즈니스 로직을 위한 상태 확장
6. **타임 트래블**: 이전 상태로 되돌아가기 및 대체 경로 탐색

## 🔑 주요 개념

- **StateGraph**: 상태 기계로 챗봇 구조를 정의
- **Node**: 작업 단위 (함수)
- **Edge**: 노드 간 전환
- **State**: 그래프 전체에서 공유되는 데이터
- **Checkpointing**: 상태 저장 및 복원
- **Tool**: 외부 기능 통합
- **Human-in-the-Loop**: 인간 검토/승인 프로세스

## 📋 사전 준비사항

- Python 3.11 이상
- OpenAI API 키
- Tavily Search API 키 (Part 2에서 필요)
- LangSmith API 키 (선택사항, 추적용)

## 환경 설정

먼저 필요한 패키지를 설치하고 환경을 설정합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Tutorial")

---

# Part 1: 기본 챗봇 구축 🤖

이 섹션에서는 LangGraph를 사용하여 기본적인 챗봇을 구축합니다. 이 챗봇은 이후 튜토리얼에서 점진적으로 더 정교한 기능을 추가할 기반이 됩니다.

## 핵심 개념

### StateGraph
- 상태(State)로 챗봇의 구조를 정의
- **노드(Nodes)**: LLM을 호출하거나 기능을 실행하는 단위
- **엣지(Edges)**: 노드 간 전환 방법을 지정

### State
- 그래프의 스키마와 리듀서 함수를 포함
- 메시지 리스트를 관리하며, 새 메시지는 기존 리스트에 추가됨

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# State 정의: 챗봇의 상태를 나타내는 타입
class State(TypedDict):
    """챗봇의 상태를 정의하는 타입

    messages: 대화 메시지 리스트
    - add_messages 함수를 통해 새 메시지가 추가됨 (덮어쓰기가 아닌 추가)
    """

    messages: Annotated[list, add_messages]


# StateGraph 생성
graph_builder = StateGraph(State)

print("✅ StateGraph 생성 완료!")
print("📌 State는 messages 키를 가지며, add_messages 리듀서를 사용합니다.")

### LLM 선택 및 설정

챗봇의 두뇌 역할을 할 언어 모델을 선택합니다.

In [ ]:
# LLM 선택
from langchain_openai import ChatOpenAI

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

print(f"✅ LLM 설정 완료: {llm.model_name}")

### 챗봇 노드 추가

**노드(Node)** 는 작업 단위이며, 일반적으로 함수로 구현됩니다.

In [ ]:
def chatbot(state: State):
    """챗봇 노드 함수

    현재 상태의 메시지를 받아 LLM에 전달하고,
    응답을 새 메시지로 추가하여 반환합니다.
    """
    # LLM을 호출하여 응답 생성
    response = llm.invoke(state["messages"])

    # 응답을 메시지 리스트에 추가하여 반환
    return {"messages": [response]}


# 그래프에 노드 추가
# 첫 번째 인자: 노드의 고유 이름
# 두 번째 인자: 노드가 사용될 때 호출될 함수
graph_builder.add_node("chatbot", chatbot)

print("✅ 챗봇 노드 추가 완료!")

### 진입점(Entry Point)과 종료점(Exit Point) 추가

그래프가 어디서 시작하고 끝나는지 정의합니다.

In [ ]:
# 진입점: 그래프 실행이 시작되는 지점
graph_builder.add_edge(START, "chatbot")

# 종료점: 그래프 실행이 끝나는 지점
graph_builder.add_edge("chatbot", END)

print("✅ 진입점과 종료점 설정 완료!")
print("📌 실행 흐름: START → chatbot → END")

### 그래프 컴파일

그래프를 실행하기 전에 컴파일해야 합니다.

In [ ]:
# 그래프 컴파일
graph = graph_builder.compile()

print("✅ 그래프 컴파일 완료!")

### 그래프 시각화

In [ ]:
from langchain_teddynote.graphs import visualize_graph

# 그래프 시각화
visualize_graph(graph)

### 챗봇 실행

이제 챗봇과 대화해봅시다!

In [ ]:
from langchain_teddynote.messages import stream_graph
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage

# 질문 입력
user_input = "안녕하세요! LangGraph에 대해 알려주세요."

# Config 설정(recursion_limit: 재귀 깊이 제한, thread_id: 스레드 아이디)
config = RunnableConfig(recursion_limit=20, thread_id="abc123")

In [ ]:
inputs = {
    "messages": [HumanMessage(content="안녕하세요! LangGraph에 대해 알려주세요.")]
}

# 그래프 스트리밍
stream_graph(graph, inputs=inputs, config=config)

---

# Part 2: 도구(Tools) 추가 🔧

챗봇이 "기억"에서 답할 수 없는 질문을 처리하기 위해 웹 검색 도구를 통합합니다. 이를 통해 챗봇이 실시간 정보를 찾아 더 나은 응답을 제공할 수 있습니다.

## 핵심 개념

- **Tool Binding**: LLM에 도구 사용 방법 알려주기
- **Tool Node**: 도구를 실행하는 노드
- **Conditional Edges**: 조건에 따라 다른 노드로 라우팅

In [ ]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition

# Tavily 검색 도구 설정
tool = TavilySearch(max_results=2)
tools = [tool]

# 도구 테스트
result = tool.invoke("LangGraph란 무엇인가요?")
print(f"검색 결과 수: {len(result['results'])}개")
print(f"첫 번째 결과 제목: {result['results'][0]['title']}")

### 도구를 사용하는 그래프 구성

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# State 정의 (동일)
class State(TypedDict):
    messages: Annotated[list, add_messages]


# 새로운 그래프 빌더 생성
builder = StateGraph(State)

# LLM에 도구 바인딩 - LLM이 도구를 사용할 수 있도록 설정
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    """도구를 사용할 수 있는 챗봇 노드"""
    # 도구가 바인딩된 LLM 호출
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


# 노드 추가
builder.add_node("chatbot", chatbot)

# ToolNode 추가 - 도구를 실행하는 노드
tool_node = ToolNode(tools=tools)
builder.add_node("tools", tool_node)

### 조건부 엣지(Conditional Edges) 추가

챗봇이 도구를 호출해야 할지 직접 응답할지 결정합니다.

### 🔍 tools_condition 함수 상세 설명

`tools_condition`은 LangGraph의 사전 정의된 조건 함수로, LLM의 응답에 도구 호출이 포함되어 있는지 확인합니다.

#### 동작 방식

```python
def tools_condition(state) -> Literal["tools", "__end__"]: 
    """마지막 메시지에 tool_calls가 있는지 확인하여 라우팅을 결정합니다."""
    
    # 1. 상태에서 마지막 메시지 추출
    if isinstance(state, list):
        ai_message = state[-1]
    elif isinstance(state, dict) and "messages" in state:
        ai_message = state["messages"][-1]
    
    # 2. 도구 호출 여부 확인
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"  # 도구 노드로 라우팅
    
    return "__end__"  # 종료
```

#### 핵심 포인트

1. **자동 판단**: LLM이 도구를 호출해야 한다고 판단하면 `tool_calls` 필드가 생성됩니다
2. **조건부 라우팅**: 
   - 도구 호출이 있으면 → `"tools"` 노드로 이동
   - 도구 호출이 없으면 → `"__end__"`로 이동 (그래프 종료)
3. **유연한 상태 처리**: 리스트, 딕셔너리, BaseModel 등 다양한 상태 형식 지원

#### 실제 사용 예시

```python
# LLM이 검색이 필요하다고 판단한 경우
# ai_message.tool_calls = [{"name": "tavily_search", "args": {"query": "..."}}, ...]
# → tools_condition은 "tools"를 반환

# LLM이 직접 답변할 수 있다고 판단한 경우
# ai_message.tool_calls = []
# → tools_condition은 "__end__"를 반환
```

In [ ]:
# 조건부 엣지 추가
# tools_condition은 메시지에 tool_calls가 있으면 "tools"로,
# 없으면 END로 라우팅합니다
builder.add_conditional_edges(
    "chatbot",
    tools_condition,  # 사전 정의된 조건 함수 사용
)

# 도구 실행 후 다시 챗봇으로 돌아가기
builder.add_edge("tools", "chatbot")

# 시작점 설정
builder.add_edge(START, "chatbot")

# 그래프 컴파일
graph_with_tools = builder.compile()

그래프 시각화

In [ ]:
# 그래프 시각화
visualize_graph(graph_with_tools)

### 도구를 사용하는 챗봇 테스트

In [ ]:
from langchain_teddynote.messages import stream_graph

stream_graph(
    graph_with_tools,
    inputs={
        "messages": [HumanMessage(content="2025년 LangGraph 사용 사례 알려주세요.")]
    },
    config=config,
)

---

# Part 3: 메모리 추가 💾

챗봇이 이전 대화 내용을 기억할 수 있도록 **체크포인팅(Checkpointing)** 을 추가합니다. 이를 통해 멀티턴 대화가 가능해집니다.

## 핵심 개념

- **Checkpointer**: 상태를 저장하고 복원하는 메커니즘
- **Thread ID**: 대화 세션을 구분하는 식별자
- **Persistent State**: 여러 호출에 걸쳐 유지되는 상태

In [ ]:
from typing import Any
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.store.base import BaseStore
from langchain_openai import ChatOpenAI
from langchain_teddynote.memory import create_memory_extractor
import uuid

model = ChatOpenAI(model="gpt-4.1", temperature=0)
memory_extractor = create_memory_extractor(model="gpt-4.1")


def call_model(
    state: MessagesState,
    config: RunnableConfig,
    *,
    store: BaseStore,
) -> dict[str, Any]:
    """Call the LLM model and manage user memory.

    Args:
        state (MessagesState): The current state containing messages.
        config (RunnableConfig): The runnable configuration.
        store (BaseStore): The memory store.
    """
    # 마지막 메시지에서 user_id 추출
    user_id = config["configurable"]["user_id"]
    namespace = ("memories", user_id)

    print(namespace)

    # 유저의 메모리 검색
    memories = store.search(namespace, query=str(state["messages"][-1].content))
    info = "\n".join([f"{memory.key}: {memory.value}" for memory in memories])
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

    # 사용자가 기억 요청 시 메모리 저장
    last_message = state["messages"][-1]
    if "remember" in last_message.content.lower():
        result = memory_extractor.invoke({"input": str(state["messages"][-1].content)})
        for memory in result.memories:
            print(memory)
            print("-" * 100)
            store.put(namespace, str(uuid.uuid4()), {memory.key: memory.value})

    # LLM 호출
    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": response}

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

# 그래프 빌드
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")

# 메모리 체크포인터 생성
# 실제 프로덕션에서는 PostgresSaver 사용 권장
memory_saver = InMemorySaver()
memory_store = InMemoryStore()

# 그래프 컴파일
graph_with_memory = builder.compile(
    checkpointer=memory_saver,
    store=memory_store,
)

In [ ]:
from langchain_teddynote.messages import stream_graph


def run_graph(
    msg,
    thread_id="default",
    user_id="default",
):
    config = {
        "configurable": {
            "thread_id": thread_id + user_id,
            "user_id": user_id,
        }
    }
    print(f"\n[유저🙋] {msg}")
    stream_graph(
        graph_with_memory,
        inputs={"messages": [{"role": "user", "content": msg}]},
        config=config,
    )
    print()

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph("안녕? 내 이름은 테디야", "1", "someone")

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph("내 이름이 뭐라고?", "1", "someone")

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph("내 이름이 뭐라고?", "2", "someone")

`remember` 라는 키워드를 입력하여 Long-term memory에 정보 저장

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph("내 이름이 테디야 remember", "2", "someone")

이렇게 저장한 Long-term memory를 활용할 때는 thread_id 가 바뀌어도 주요 정보를 기억하고 있습니다.

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph("내 이름이 뭐라고 했더라?", "3", "someone")

In [ ]:
# 메시지, thread_id, user_id 전달
run_graph(
    "내 직업은 AI Engineer 야. 내 취미는 Netflix 보기 야. remember", "4", "someone"
)

In [ ]:
# 다른 스레드에서 실행
run_graph("내 이름, 직업, 취미 알려줘", "100", "someone")

In [ ]:
# 다른 user_id 로 실행한 경우
run_graph("내 이름, 직업, 취미 알려줘", "100", "other")

### State 확인

`get_state` 함수를 사용하여 저장된 상태를 확인할 수 있습니다.

In [ ]:
# 임의의 Config 설정
config = {
    "configurable": {
        "thread_id": "100" + "someone",
        "user_id": "someone",
    }
}

# 현재 상태 가져오기
snapshot = graph_with_memory.get_state(config)

print("📊 현재 상태 정보:")
print(f"- 메시지 수: {len(snapshot.values['messages'])}개")
print(f"- 체크포인트 ID: {snapshot.config['configurable']['checkpoint_id']}")

# 최근 메시지 몇 개 표시
print("\n[최근 메시지]")
for msg in snapshot.values["messages"]:
    role = msg.type if hasattr(msg, "type") else "unknown"
    content = msg.content if hasattr(msg, "content") else str(msg)
    print(f"  [{role}]: {content}")

---

# Part 4: Human-in-the-Loop 🙋

에이전트가 신뢰할 수 없거나 중요한 결정을 내릴 때 인간의 입력이 필요할 수 있습니다. LangGraph의 **interrupt** 기능을 사용하여 실행을 일시 중지하고 인간의 피드백을 받을 수 있습니다.

## 핵심 개념

- **interrupt**: 실행을 일시 중지하는 함수
- **Command**: 실행을 재개하고 데이터를 전달하는 객체
- **Human Approval**: 인간 검토/승인 프로세스

In [ ]:
from langchain_core.tools import tool
from langgraph.types import Command, interrupt


@tool
def human_assistance(query: str) -> str:
    """Request assistance from an expert(human)."""
    # interrupt를 호출하여 실행 일시 중지
    # 사람의 응답을 기다림
    human_response = interrupt({"query": query})

    # 사람의 응답 반환
    return human_response["data"]

### Human-in-the-Loop가 있는 그래프 구성

In [ ]:
# 도구 리스트 업데이트
tools_with_human = [human_assistance]

# 새로운 그래프 구성
graph_builder_hitl = StateGraph(State)

# LLM에 도구 바인딩
llm_with_human_tools = llm.bind_tools(tools_with_human)


def chatbot_with_human(state: State):
    """Human Interuption 요청할 수 있는 챗봇"""
    message = llm_with_human_tools.invoke(state["messages"])

    # interrupt 중 병렬 도구 호출 방지
    # (재개 시 도구 호출이 반복되는 것을 방지)
    if hasattr(message, "tool_calls"):
        assert (
            len(message.tool_calls) <= 1
        ), "병렬 도구 호출은 interrupt와 함께 사용할 수 없습니다"

    return {"messages": [message]}


# 노드 추가
graph_builder_hitl.add_node("chatbot_with_human", chatbot_with_human)

# ToolNode 추가
tool_node_hitl = ToolNode(tools=tools_with_human)
graph_builder_hitl.add_node("tools", tool_node_hitl)

# 엣지 추가
graph_builder_hitl.add_conditional_edges("chatbot_with_human", tools_condition)
graph_builder_hitl.add_edge("tools", "chatbot_with_human")
graph_builder_hitl.add_edge(START, "chatbot_with_human")

# 메모리와 함께 컴파일
memory_hitl = InMemorySaver()
graph_hitl = graph_builder_hitl.compile(checkpointer=memory_hitl)

# 그래프 시각화
visualize_graph(graph_hitl)

### Human-in-the-Loop 테스트

In [ ]:
from langchain_teddynote.messages import random_uuid

# 인간 지원을 요청하는 메시지
user_input = "LangGraph 잘하고 싶은데, 사람에게 조언을 듣고 싶어요."
config_hitl = {"configurable": {"thread_id": random_uuid()}}

print(f"User: {user_input}\n")

stream_graph(
    graph_hitl,
    inputs={"messages": [HumanMessage(content=user_input)]},
    config=config_hitl,
)

In [ ]:
# 상태 확인 - 어느 노드에서 중단되었는지 확인
snapshot = graph_hitl.get_state(config_hitl)
print(f"\n📊 현재 상태:")
print(f"  다음 실행할 노드: {snapshot.next}")
print(f"  체크포인트 ID: {snapshot.config['configurable']['checkpoint_id']}")

In [ ]:
# 인간의 응답으로 실행 재개
human_response = """## 전문가의 조언: 
- YouTube 테디노트: https://www.youtube.com/c/teddynote
- 고급 개발자 강의 [패스트캠퍼스 RAG 비법노트](https://fastcampus.co.kr/data_online_teddy)
"""

# Command 객체로 재개
human_command = Command(resume={"data": human_response})

print(f"\n💡 사람의 응답: {human_response}\n")

# 재개
stream_graph(graph_hitl, inputs=human_command, config=config_hitl)

---

# Part 5: 상태 커스터마이징 🎨

메시지 리스트 외에 추가 필드를 상태에 추가하여 복잡한 동작을 정의할 수 있습니다. 예를 들어, 특정 정보를 저장하고 인간의 검토를 받는 워크플로우를 구현해봅시다.

## 핵심 개념

- **Custom State Fields**: 상태에 커스텀 필드 추가
- **State Updates from Tools**: 도구 내부에서 상태 업데이트
- **Manual State Updates**: 수동으로 상태 변경

In [ ]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId


# 확장된 State 정의
class CustomState(TypedDict):
    """커스텀 필드가 추가된 상태"""

    messages: Annotated[list, add_messages]
    human_feedback: str  # 사람의 피드백

### 상태를 업데이트하는 도구

In [ ]:
@tool
def human_review(
    human_feedback, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request human review for information."""
    # 인간에게 검토 요청
    human_response = interrupt(
        {"question": "이 정보가 맞나요?", "human_feedback": human_feedback}
    )

    feedback = human_response.get("human_feedback", "")

    if feedback.strip() == "":
        # 사용자가 AI 의 답변에 동의하는 경우
        return Command(
            update={
                "messages": [ToolMessage(human_response, tool_call_id=tool_call_id)]
            }
        )
    else:
        # 사용자가 AI 의 답변에 동의하지 않는 경우
        corrected_information = f"# 사용자에 의해 수정된 피드백: {feedback}"
        return Command(
            update={
                "messages": [
                    ToolMessage(corrected_information, tool_call_id=tool_call_id)
                ]
            }
        )

### 커스텀 상태를 사용하는 그래프

In [ ]:
# 도구 리스트
tools_custom = [human_review]

# 새로운 그래프 구성
custom_graph_builder = StateGraph(CustomState)  # CustomState 사용

# LLM에 도구 바인딩
llm_with_custom_tools = llm.bind_tools(tools_custom)


def chatbot_custom(state: CustomState):
    """커스텀 상태를 사용하는 챗봇"""
    message = llm_with_custom_tools.invoke(state["messages"])

    if hasattr(message, "tool_calls"):
        assert len(message.tool_calls) <= 1

    return {"messages": [message]}


# 노드와 엣지 추가
custom_graph_builder.add_node("chatbot", chatbot_custom)
tool_node_custom = ToolNode(tools=tools_custom)
custom_graph_builder.add_node("tools", tool_node_custom)

custom_graph_builder.add_conditional_edges("chatbot", tools_condition)
custom_graph_builder.add_edge("tools", "chatbot")
custom_graph_builder.add_edge(START, "chatbot")

# 컴파일
memory_custom = InMemorySaver()
custom_graph = custom_graph_builder.compile(checkpointer=memory_custom)

그래프를 시각화 합니다.

In [ ]:
# 그래프 시각화
visualize_graph(custom_graph)

### 커스텀 상태 테스트

In [ ]:
# LangGraph의 출시일을 조사하고 검토 요청
user_input = (
    "2024년 노벨 문학상 수상자가 누구인지 조사해주세요. "
    "답을 찾으면 `human_review` 도구를 사용해서 검토를 요청하세요."
)

custom_config = RunnableConfig(configurable={"thread_id": random_uuid()})

print(f"User: {user_input}\n")

# 실행 (interrupt에서 중단될 것임)
stream_graph(
    custom_graph,
    inputs={"messages": [HumanMessage(content=user_input)]},
    config=custom_config,
)

In [ ]:
from langchain_teddynote.messages import display_message_tree

# 최신 메시지 가져오기
last_message = custom_graph.get_state(custom_config).values["messages"][-1]

# 최신 메시지 tree 구조로 표시
display_message_tree(last_message)

In [ ]:
# AI 가 작성한 내용
print(last_message.tool_calls[0]["args"]["human_feedback"])

In [ ]:
# 인간의 검토 응답으로 재개
human_command = Command(
    resume={"human_feedback": "2024년 노벨 문학상 수상자는 대한민국의 한강 작가입니다."}
)

stream_graph(custom_graph, inputs=human_command, config=custom_config)

---

# Part 6: 타임 트래블 ⏰

LangGraph의 **타임 트래블** 기능을 사용하면 이전 체크포인트로 돌아가서 다른 경로를 탐색할 수 있습니다. 이는 디버깅, 실험, 대화형 애플리케이션에 매우 유용합니다.

## 핵심 개념

- **State History**: 모든 체크포인트의 기록
- **Checkpoint ID**: 특정 시점의 식별자
- **Rewind**: 이전 상태로 되돌리기
- **Resume**: 특정 체크포인트에서 재개

### 타임 트래블을 위한 그래프 준비

In [ ]:
# 타임 트래블 테스트를 위한 간단한 그래프
graph_builder = StateGraph(State)

# 도구와 LLM 설정
tools = [TavilySearch(max_results=2)]
llm_with_tools_tt = llm.bind_tools(tools)


def chatbot_tt(state: State):
    """타임 트래블 테스트용 챗봇"""
    return {"messages": [llm_with_tools_tt.invoke(state["messages"])]}


# 그래프 구성
graph_builder.add_node("chatbot", chatbot_tt)
tool_node_tt = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node_tt)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# 메모리와 함께 컴파일
memory_tt = InMemorySaver()
time_travel_graph = graph_builder.compile(checkpointer=memory_tt)

시각화를 수행합니다.

In [ ]:
# 시각화
visualize_graph(time_travel_graph)

### 여러 단계의 대화 생성

In [ ]:
time_travel_config = RunnableConfig(configurable={"thread_id": "time-travel-1"})

# 첫 번째 대화
stream_graph(
    time_travel_graph,
    inputs={"messages": [HumanMessage(content="테디노트에 대해서 조사 좀 해주세요.")]},
    config=time_travel_config,
)

In [ ]:
# 두 번째 대화
stream_graph(
    time_travel_graph,
    inputs={
        "messages": [
            HumanMessage(content="테디노트 온라인 강의 주소를 조사해 해주세요.")
        ]
    },
    config=time_travel_config,
)

### 상태 히스토리 탐색

In [ ]:
# 전체 상태 히스토리 확인
print("📜 상태 히스토리 (최신순):")
print("=" * 80)

# to_replay 변수 초기화
to_replay = None

for i, state in enumerate(time_travel_graph.get_state_history(time_travel_config)):
    print(f"\n[체크포인트 {i}]")
    print(f"  다음 노드: {state.next}")
    print(f"  체크포인트 ID: {state.config['configurable']['checkpoint_id']}")

    if len(state.values["messages"]) == 6 and to_replay is None:
        print("  ⭐ 이 상태로 되돌아갈 예정")
        display_message_tree(state.values["messages"][-1])
        to_replay = state


print("\n" + "=" * 80)

### 특정 체크포인트에서 재개

In [ ]:
display_message_tree(to_replay.values["messages"][-1])

`update_tool_call` 을 활용하여 검색할 쿼리를 수정합니다.

In [ ]:
from langchain_teddynote.tools import update_tool_call

# 사용 예시:
updated_message = update_tool_call(
    to_replay.values["messages"][-1],
    tool_name="tavily_search",
    tool_args={"query": "테디노트 강의 site:fastcampus.co.kr", "search_depth": "basic"},
)

In [ ]:
# 변경하기 전의 message
display_message_tree(to_replay.values["messages"][-1])

In [ ]:
# 변경한 이후의 메시지 트리
display_message_tree(updated_message)

In [ ]:
# 변경된 메시지를 update_state 로 업데이트
updated_state = time_travel_graph.update_state(
    values={"messages": [updated_message]}, config=to_replay.config
)

업데이트된 메시지를 스트리밍 합니다. 여기서 `inputs` 는 `None` 으로 주고, `config` 는 `updated_state` 로 주어 Replay 합니다.

In [ ]:
# 업데이트된 메시지를 스트리밍 합니다.
stream_graph(time_travel_graph, inputs=None, config=updated_state)